In [4]:
# Importeer de benodigde bibliotheken
import pandas as pd
from freqtrade.strategy.interface import IStrategy
from pandas import DataFrame
import talib.abstract as ta
import freqtrade.vendor.qtpylib.indicators as qtpylib
from scipy.spatial.distance import cosine
import numpy as np

In [5]:
# Load data
df = pd.read_json("BTC_USDT-15m.json")

In [6]:
df.columns=['date','open','high','low','close','volume']
df['date']=(pd.to_datetime(df['date'],unit='ms'))
df.tail(5)

,date,open,high,low,close,volume
127189,2021-04-08 07:00:00,57125.98,57333.85,57059.01,57162.86,622.845366
127190,2021-04-08 07:15:00,57162.86,57204.92,57005.72,57080.59,431.831293
127191,2021-04-08 07:30:00,57080.59,57173.34,56995.00,57169.98,429.680958
127192,2021-04-08 07:45:00,57169.98,57200.00,57011.11,57025.71,354.884159
127193,2021-04-08 08:00:00,57023.63,57160.00,57014.58,57105.21,375.847461


## Add indicators

In [7]:
df['rsi'] = ta.RSI(df, timeperiod=7)
stoch = ta.STOCH(
    df,
    fastk_period=14,
    slowk_period=3,
    slowk_matype=0,
    slowd_period=3,
    slowd_matype=0,
)
df['slowd'] = stoch['slowd']
df['slowk'] = stoch['slowk']
        

In [8]:
print(df)

                      date      open      high       low     close  \
0      2017-08-17 04:00:00   4261.48   4280.56   4261.48   4261.48   
1      2017-08-17 04:15:00   4261.48   4270.41   4261.32   4261.45   
2      2017-08-17 04:30:00   4280.00   4310.07   4267.99   4310.07   
3      2017-08-17 04:45:00   4310.07   4313.62   4291.37   4308.83   
4      2017-08-17 05:00:00   4308.83   4328.69   4304.31   4304.31   
...                    ...       ...       ...       ...       ...   
127189 2021-04-08 07:00:00  57125.98  57333.85  57059.01  57162.86   
127190 2021-04-08 07:15:00  57162.86  57204.92  57005.72  57080.59   
127191 2021-04-08 07:30:00  57080.59  57173.34  56995.00  57169.98   
127192 2021-04-08 07:45:00  57169.98  57200.00  57011.11  57025.71   
127193 2021-04-08 08:00:00  57023.63  57160.00  57014.58  57105.21   

            volume        rsi      slowd      slowk  
0         2.189061        NaN        NaN        NaN  
1         9.119865        NaN        NaN        NaN

In [9]:
df["rsi_sma"] = df["rsi"].rolling(window=10).mean()
print(df[['date','rsi','rsi_sma']])

                      date        rsi    rsi_sma
0      2017-08-17 04:00:00        NaN        NaN
1      2017-08-17 04:15:00        NaN        NaN
2      2017-08-17 04:30:00        NaN        NaN
3      2017-08-17 04:45:00        NaN        NaN
4      2017-08-17 05:00:00        NaN        NaN
...                    ...        ...        ...
127189 2021-04-08 07:00:00  68.468925  70.406321
127190 2021-04-08 07:15:00  59.537786  68.853943
127191 2021-04-08 07:30:00  65.278955  68.158028
127192 2021-04-08 07:45:00  51.515624  65.899979
127193 2021-04-08 08:00:00  57.303046  64.053205

[127194 rows x 3 columns]


## Nieuw idee dat misschien gaat werken:

Timeframe 15m:

* Eerst dient slow stoch (14-3-3) onder 20 een k/d bullish crossover te krijgen
* Vervolgens dient rsi7 boven rsima10 te komen
* Tot slot dient binnen 5 candles de closeprijs boven sma21 te sluiten
* Dan instappen met een R:R van 1:1. Stoploss is lowest low + 10 pips van entry candle die boven de sma21 is gekomen


In [10]:
# True if slowd and slowk are both below 30
df['stoch_sub20']=(df['slowd']<30)&(df['slowk']<30)
print(df[['date','slowd','slowk','stoch_sub20']].tail(55))

                      date      slowd      slowk  stoch_sub20
127139 2021-04-07 18:30:00  19.898856  24.545838         True
127140 2021-04-07 18:45:00  17.616216  11.651282         True
127141 2021-04-07 19:00:00  17.261233  15.586578         True
127142 2021-04-07 19:15:00  13.873328  14.382125         True
127143 2021-04-07 19:30:00  17.953181  23.890839         True
127144 2021-04-07 19:45:00  24.893134  36.406439        False
127145 2021-04-07 20:00:00  33.474304  40.125635        False
127146 2021-04-07 20:15:00  38.484408  38.921148        False
127147 2021-04-07 20:30:00  40.905800  43.670616        False
127148 2021-04-07 20:45:00  46.499615  56.907081        False
127149 2021-04-07 21:00:00  57.135557  70.828973        False
127150 2021-04-07 21:15:00  68.606152  78.082401        False
127151 2021-04-07 21:30:00  77.471418  83.502879        False
127152 2021-04-07 21:45:00  82.979245  87.352455        False
127153 2021-04-07 22:00:00  86.248234  87.889367        False
127154 2

In [11]:
# True if slowk is greater than slowd
df['stoch_cross'] = df['slowk'] > df['slowd']
print(df[['date','slowk','slowd','stoch_cross']].tail(55))

                      date      slowk      slowd  stoch_cross
127139 2021-04-07 18:30:00  24.545838  19.898856         True
127140 2021-04-07 18:45:00  11.651282  17.616216        False
127141 2021-04-07 19:00:00  15.586578  17.261233        False
127142 2021-04-07 19:15:00  14.382125  13.873328         True
127143 2021-04-07 19:30:00  23.890839  17.953181         True
127144 2021-04-07 19:45:00  36.406439  24.893134         True
127145 2021-04-07 20:00:00  40.125635  33.474304         True
127146 2021-04-07 20:15:00  38.921148  38.484408         True
127147 2021-04-07 20:30:00  43.670616  40.905800         True
127148 2021-04-07 20:45:00  56.907081  46.499615         True
127149 2021-04-07 21:00:00  70.828973  57.135557         True
127150 2021-04-07 21:15:00  78.082401  68.606152         True
127151 2021-04-07 21:30:00  83.502879  77.471418         True
127152 2021-04-07 21:45:00  87.352455  82.979245         True
127153 2021-04-07 22:00:00  87.889367  86.248234         True
127154 2

In [12]:
# True if stoch_sub20 == true and stoch_cross == true
# Means that a cross has been made below 20
df['stoch_signal'] = (df['stoch_sub20'] == True)&(df['stoch_cross'] == True)
print(df[['date','stoch_sub20','stoch_cross','stoch_signal']].tail(55))

                      date  stoch_sub20  stoch_cross  stoch_signal
127139 2021-04-07 18:30:00         True         True          True
127140 2021-04-07 18:45:00         True        False         False
127141 2021-04-07 19:00:00         True        False         False
127142 2021-04-07 19:15:00         True         True          True
127143 2021-04-07 19:30:00         True         True          True
127144 2021-04-07 19:45:00        False         True         False
127145 2021-04-07 20:00:00        False         True         False
127146 2021-04-07 20:15:00        False         True         False
127147 2021-04-07 20:30:00        False         True         False
127148 2021-04-07 20:45:00        False         True         False
127149 2021-04-07 21:00:00        False         True         False
127150 2021-04-07 21:15:00        False         True         False
127151 2021-04-07 21:30:00        False         True         False
127152 2021-04-07 21:45:00        False         True         F

In [13]:
# True if rsi is above rsi_sma
df['rsi_cross']=df['rsi']>df['rsi_sma']
print(df[['rsi','rsi_sma','rsi_cross']].tail(55))

              rsi    rsi_sma  rsi_cross
127139  44.679875  39.109292       True
127140  36.461499  38.806873      False
127141  32.038607  36.541339      False
127142  40.536359  35.654657       True
127143  42.606947  35.724959       True
127144  52.632022  37.824321       True
127145  47.183843  39.899634       True
127146  39.923757  41.310231      False
127147  52.282292  41.810895       True
127148  57.132014  44.547722       True
127149  53.098923  45.389626       True
127150  58.828914  47.626368       True
127151  60.750617  50.497569       True
127152  52.162574  51.660190       True
127153  54.588824  52.858378       True
127154  58.198944  53.415070       True
127155  66.869834  55.383669       True
127156  60.416075  57.432901       True
127157  61.946312  58.399303       True
127158  56.021376  58.288239      False
127159  45.695038  57.547851      False
127160  32.757893  54.940749      False
127161  43.628357  53.228523      False
127162  43.959654  52.408231      False


In [14]:
# Check if the stoch_signal_happened when the rsi_cross occured.
# This way we have signal 1 that triggers when rsi_cross and stoch_cross below 30 happened.
df['entry_signal_1']=(df['rsi_cross'] == True)&(df['stoch_signal'] == True)
print(df[['date','rsi_cross','stoch_signal','entry_signal_1']].tail(55))

                      date  rsi_cross  stoch_signal  entry_signal_1
127139 2021-04-07 18:30:00       True          True            True
127140 2021-04-07 18:45:00      False         False           False
127141 2021-04-07 19:00:00      False         False           False
127142 2021-04-07 19:15:00       True          True            True
127143 2021-04-07 19:30:00       True          True            True
127144 2021-04-07 19:45:00       True         False           False
127145 2021-04-07 20:00:00       True         False           False
127146 2021-04-07 20:15:00      False         False           False
127147 2021-04-07 20:30:00       True         False           False
127148 2021-04-07 20:45:00       True         False           False
127149 2021-04-07 21:00:00       True         False           False
127150 2021-04-07 21:15:00       True         False           False
127151 2021-04-07 21:30:00       True         False           False
127152 2021-04-07 21:45:00       True         Fa

In [15]:
# Check if closeprice is above sma21
df['sma'] = ta.SMA(df, timeperiod=21)
df['entry_signal_2'] = df['close'] > df['sma']
print(df[['date','close','sma','entry_signal_2']].tail(55))

                      date     close           sma  entry_signal_2
127139 2021-04-07 18:30:00  56152.68  56490.049524           False
127140 2021-04-07 18:45:00  55859.13  56444.693810           False
127141 2021-04-07 19:00:00  55670.29  56392.961905           False
127142 2021-04-07 19:15:00  55860.98  56348.734286           False
127143 2021-04-07 19:30:00  55908.14  56304.630952           False
127144 2021-04-07 19:45:00  56153.83  56274.353333           False
127145 2021-04-07 20:00:00  56014.62  56243.620952           False
127146 2021-04-07 20:15:00  55805.00  56194.280476           False
127147 2021-04-07 20:30:00  56107.43  56169.049524           False
127148 2021-04-07 20:45:00  56249.99  56161.204762            True
127149 2021-04-07 21:00:00  56158.67  56147.262857            True
127150 2021-04-07 21:15:00  56312.99  56140.295714            True
127151 2021-04-07 21:30:00  56366.00  56136.341429            True
127152 2021-04-07 21:45:00  56205.73  56114.372381            

In [16]:
# Check if signal 1 happened within the last 5 candles.
df['signal_1_happened']=(df['entry_signal_1'].shift(+5)==True)|(df['entry_signal_1'].shift(+4)==True)|(df['entry_signal_1'].shift(+3)==True)|(df['entry_signal_1'].shift(+2)==True)|(df['entry_signal_1'].shift(+1)==True)|(df['entry_signal_1']==True)
print(df[['date','entry_signal_1','signal_1_happened']].tail(55))


                      date  entry_signal_1  signal_1_happened
127139 2021-04-07 18:30:00            True               True
127140 2021-04-07 18:45:00           False               True
127141 2021-04-07 19:00:00           False               True
127142 2021-04-07 19:15:00            True               True
127143 2021-04-07 19:30:00            True               True
127144 2021-04-07 19:45:00           False               True
127145 2021-04-07 20:00:00           False               True
127146 2021-04-07 20:15:00           False               True
127147 2021-04-07 20:30:00           False               True
127148 2021-04-07 20:45:00           False               True
127149 2021-04-07 21:00:00           False              False
127150 2021-04-07 21:15:00           False              False
127151 2021-04-07 21:30:00           False              False
127152 2021-04-07 21:45:00           False              False
127153 2021-04-07 22:00:00           False              False
127154 2

In [17]:
# FInally check if signal 2 occurs when signal 1 happened within the last 5 candles
df['final_signal']=(df['entry_signal_2']==True)&(df['signal_1_happened']==True)
# print(df[['date','entry_signal_2','signal_1_happened','final_signal']].loc[350:405, : ])
# Print all rows where final signal == True (limit to last 50)
print(df[['date','final_signal']].loc[df['final_signal'] == True].tail(50))

                      date  final_signal
124774 2021-03-14 03:15:00          True
124775 2021-03-14 03:30:00          True
124776 2021-03-14 03:45:00          True
124934 2021-03-15 19:15:00          True
125072 2021-03-17 05:45:00          True
125073 2021-03-17 06:00:00          True
125074 2021-03-17 06:15:00          True
125075 2021-03-17 06:30:00          True
125205 2021-03-18 15:00:00          True
125206 2021-03-18 15:15:00          True
125207 2021-03-18 15:30:00          True
125208 2021-03-18 15:45:00          True
125366 2021-03-20 07:15:00          True
125367 2021-03-20 07:30:00          True
125541 2021-03-22 03:00:00          True
125542 2021-03-22 03:15:00          True
125543 2021-03-22 03:30:00          True
125656 2021-03-23 07:45:00          True
125657 2021-03-23 08:00:00          True
125728 2021-03-24 01:45:00          True
125729 2021-03-24 02:00:00          True
125730 2021-03-24 02:15:00          True
125916 2021-03-26 00:45:00          True
125917 2021-03-2

Bij backtests over een maandje blijkt de strategie niet helemaal lekker te werken. De maand is wel erg negatief, dus misschien ligt het daar aan. Zaak is wel om juist onder deze omstandigheden een aardige winst te boeken.

In [21]:
# 
# Calculate slope - only take trades when slope is positive
df['fy1'] = df['sma'].shift(+1)
df['fy2'] = df['sma'].shift(+11)
fx1 = 1
fx2 = 11
df['fy'] = df['fy1'] - df['fy2']
df['fx'] = fx2 - fx1
df['fast_slope'] = df['fy']/df['fx']

# print(df[['date','sma','fy1','fy2','fy','fx','fast_slope']].tail(55))
print(df[['date','sma','fy','fx','fast_slope']].tail(55))

# dataframe['last_lowest'] = dataframe['close'].rolling(10).min().shift(1)


                      date           sma          fy  fx  fast_slope
127139 2021-04-07 18:30:00  56490.049524  -24.887143  10   -2.488714
127140 2021-04-07 18:45:00  56444.693810  -28.338095  10   -2.833810
127141 2021-04-07 19:00:00  56392.961905  -52.684762  10   -5.268476
127142 2021-04-07 19:15:00  56348.734286 -122.852857  10  -12.285286
127143 2021-04-07 19:30:00  56304.630952 -195.362857  10  -19.536286
127144 2021-04-07 19:45:00  56274.353333 -246.165238  10  -24.616524
127145 2021-04-07 20:00:00  56243.620952 -294.507143  10  -29.450714
127146 2021-04-07 20:15:00  56194.280476 -322.870476  10  -32.287048
127147 2021-04-07 20:30:00  56169.049524 -356.844286  10  -35.684429
127148 2021-04-07 20:45:00  56161.204762 -376.549524  10  -37.654952
127149 2021-04-07 21:00:00  56147.262857 -347.925238  10  -34.792524
127150 2021-04-07 21:15:00  56140.295714 -342.786667  10  -34.278667
127151 2021-04-07 21:30:00  56136.341429 -304.398095  10  -30.439810
127152 2021-04-07 21:45:00  56114.

In de strategie ziet bovenstaande er als volgt uit:
```
# The RSI should have a bullish cross above the rsi_ma10
        dataframe['rsi_cross']=dataframe['rsi']>dataframe['rsi_sma']
        # Slowd and Slowk should be below 30 before giving a signal
        dataframe['stoch_sub']=(dataframe['slowd']<30)&(dataframe['slowk']<30)
        # There should be a bullish crossover between slowk and slowd
        dataframe['stoch_cross'] = dataframe['slowk'] > dataframe['slowd']
        # ENTRY SIGNAL 1: When below 30 and a bullish crossover, then the first signal (stochastic) is true. 
        dataframe['entry_signal_1'] = (dataframe['stoch_sub'] == True)&(dataframe['stoch_cross'] == True)
        #  ENTRY SIGNAL 2: The stoch signal should alsoo have happened during the RSI crossover signal, which gives entry signal 2
        dataframe['entry_signal_2']=(dataframe['rsi_cross'] == True)&(dataframe['entry_signal_1'] == True)
        # The third entry signal should be given when candle closes above SMA
        dataframe['entry_signal_3'] = dataframe['close'] > dataframe['sma']
        # Prepare to check that entry signal 2 happened within the last 5 candles
        dataframe['signal_2_happened']=(dataframe['entry_signal_2'].shift(+5)==True)|(dataframe['entry_signal_2'].shift(+4)==True)|(dataframe['entry_signal_2'].shift(+3)==True)|(dataframe['entry_signal_2'].shift(+2)==True)|(dataframe['entry_signal_2'].shift(+1)==True)|(dataframe['entry_signal_2']==True)
        # The final signal is given when signal 3 occured when signal 2 happened
        dataframe['final_signal']=(dataframe['entry_signal_3']==True)&(dataframe['signal_2_happened']==True)

        print(dataframe[['date','final_signal']].loc[dataframe['final_signal'] == True].tail(50))
        return dataframe

```

In [19]:
https://www.opentechguides.com/how-to/article/pandas/193/index-slice-subset.html
How to use for specific row

    loc

df.loc[row,column]

For first row and all column

df.loc[0,:]

For first row and some specific column

df.loc[0,'column_name']

    iloc

For first row and all column

df.iloc[0,:]

For first row and some specific column i.e first three cols

df.iloc[0,0:3]


SyntaxError: invalid syntax (<ipython-input-19-7a33e7d66b03>, line 1)

## Misschien een goed idee maar nog niet weten hoe uit te voeren

Wanneer RSI of Stoch slowd onder een bepaalde waarde (rsi - 30; slowd - 30) zijn gekomen. Zet een trigger die af gaat als beiden weer boven deze waarden komen (crossup trigger).
Dus: RSI komt onder 30 -> trigger gezet; RSI komt boven 30 -> Trigger los en buy signal.
Daarna moet deze trigger weer gereset worden. Dat gebeurd als de waarde boven 50 komt.
Logisch is dat dus:
```
* Als RSI < 30 & df['rsi_trigger'].shift(+1) == 2 => df['rsi_trigger'] = 0
* Als RSI > 30 & df['rsi_trigger'].shift(+1) == 0 => df['rsi_trigger'] = 1
* Als RSI > 50 & df['rsi_trigger'].shift(+1) == 1 => df['rsi_trigger'] = 2
* Else df['rsi_trigger'] = 3
```

In [15]:
# df['below'] = (df['rsi'].iloc[:-5] > 90) & (df['rsi'].iloc[:-1] > 90)
df['below'] = (df['rsi'].iloc[:-4] < 70)&(df['rsi'].iloc[:-1] < 70)
print(df[['rsi','below']].tail(15))

              rsi  below
127179  73.462489  False
127180  75.061567  False
127181  72.238105  False
127182  74.096116  False
127183  75.770785  False
127184  62.624898   True
127185  78.987266  False
127186  65.371688   True
127187  64.902965   True
127188  66.540897   True
127189  68.468925   True
127190  59.537786  False
127191  65.278955  False
127192  51.515624  False
127193  57.303046    NaN


In [16]:
# Andere handige conditionele waarden snippet:
# Calculate positions
rsi_conditions = [
    (dataframe['rsi'] < 45),
    (dataframe['rsi'] > 75),
]
slowd_conditions = [
    (dataframe['slowd'] < 20),
    (dataframe['slowd'] > 80)
]
slowk_conditions = [
    (dataframe['slowk'] < 20),
    (dataframe['slowk'] > 80)
]
stoch_conditions = [
    (dataframe['slowk'] > dataframe['slowd']),
    (dataframe['slowk'] < dataframe['slowd'])
]
values = [1,5]
dataframe['rsi_pos']=np.select(rsi_conditions,values)
dataframe['slowd_pos']=np.select(slowd_conditions,values)
dataframe['slowk_pos']=np.select(slowk_conditions,values)
dataframe['stoch_pos']=np.select(stoch_conditions,values

SyntaxError: unexpected EOF while parsing (<ipython-input-16-2693afa8acba>, line 23)

In [ ]:
rsi_conditions = [
    (dataframe['rsi'] < 30),
    (dataframe['rsi'] < 45),
    (dataframe['rsi'] > 75),
]
values = [0,1,5]
dataframe['rsi_pos']=np.select(rsi_conditions,values)
print(df[['rsi','rsi_pos']])